In [2]:
import numpy as np
import rpy2
import rpy2.robjects as robjects
import pickle
from time import time
import os

# RPY2 is used an interconnect between Python and R. It allows
# my to run R code from python which makes this experimentation
# process smoother.
from rpy2.robjects import IntVector, FloatVector, Formula
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
numpy2ri.activate()

stats = importr('stats') # standard regression package
matching = importr('Matching') # GenMatch package
snow = importr('snow') # cluster manager

## 1. Define an Experimental Framework

### A. Cluster Compute

See the simulations file for details on this cluster compute code

In [3]:
AWS_MASTER_DNS="ip-172-31-42-147.ec2.internal"
AWS_SLAVE_1 = "ubuntu@ip-172-31-43-193.ec2.internal"
AWS_SLAVE_2 = "ubuntu@ip-172-31-81-244.ec2.internal"
AWS_MASTER_PORT_RANGE = list(range(11305, 11340))

class ClusterProvider(object):
    def __init__(self, n_nodes=8, remote_hosts=None, ports=None):
        if remote_hosts is None:
            self.cl = snow.makeSOCKcluster(["localhost"]*n_nodes)
        else:
            # Set the acceptable ports for connection
            # from the slaves
            if not ports:
                ports = AWS_MASTER_PORT_RANGE
            
            # Construct the connection string
            addresses = []
            for remote_host, n_nodes in remote_hosts:
                addresses+=[remote_host]*n_nodes
                
            self.cl = snow.makeSOCKcluster(addresses, rscript="Rscript", manual=False, snowlib="/usr/local/lib/R/site-library",
                                           port=IntVector(ports), master=AWS_MASTER_DNS, outfile="/dev/stdout", timeout=10)
    
    def get_cluster(self):
        return self.cl
    
    def kill_cluster(self):
        snow.stopCluster(self.cl)

In [73]:
# Local cluster
cluster_provider = ClusterProvider(n_nodes=8)

In [11]:
# Remote cluster
# cluster_provider = ClusterProvider(remote_hosts=[(AWS_SLAVE_1, 8)],
#                                     ports = list(range(11305, 11314)))

In [72]:
# Run this with True to kill the cluster
kill = True # termination protection
if kill:
    cluster_provider.kill_cluster()

### B. Flexible Causal Inference Methods

#### Estimators

In this file, we copy across the GenMatch and Logistic Propensity score methods implemented in the simulations file. See the Simulations notebook for details on these functions

In [6]:
# Helper function which runs logistic regression in R
# to determine propensity scores for a dataset. This is used
# in the propensity score matching method and in GenMatch
def get_propensity_scores(assignments, covariate_data):
    # Setup
    y = IntVector(assignments)
    fmla = Formula('y ~ X')
    env = fmla.environment
    
    # Run propensiy regression
    env['X'] = covariate_data
    env['y'] = y
    fit = stats.glm(fmla, family="binomial")
    
    # DEBUG: fit.rx("coefficients")
    return fit.rx2("fitted.values")

In [93]:
# 1. Logisic Regression Propensity Matching
def logistic_prop_matching_est(outcomes, assignments, covariate_data, *args, **kwargs):
    global gm_warnings
    logistic_propensity_scores = get_propensity_scores(assignments, covariate_data)
    
    # Use prop scores from the neural network regression
    # if supplied
    nn_p_scores = kwargs.get("nn_p_scores", None)
    if nn_p_scores is not None:
        if gm_warnings:
            print("Using p-scores from neural net")
        propensity_scores = nn_p_scores
    else:
        propensity_scores = logistic_propensity_scores
    
    # Run matching on prop scores
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=propensity_scores,
        replace=True)
    
    gm_warnings = False # only warn once
    return (np.array(match_out.rx2("est").rx(1,1))[0], None)

In [105]:
# 2. GenMatch Matching
def genmatch_est(outcomes, assignments, covariate_data, *args, **kwargs):
    
    # Get the singleton cluster
    cl = cluster_provider.get_cluster()
    
    # Flag on whethert or not to use propensity scores
    # in GenMatch
    if kwargs.get("genmatch_with_prop_scores", True):
        print("Genmatch running with p scores")
        
        # Parameter to allow prop scores to
        # be derived from custom data
        propensity_vars = kwargs.get("propensity_vars", None)
        if propensity_vars is None:
            print("Finding propensity score on matching vars")
            propensity_vars = covariate_data
        else:
            print("Finding propensity scores with custom vars")

        logistic_p_scores = np.array(get_propensity_scores(assignments, propensity_vars))
            
        nn_p_scores = kwargs.get("nn_p_scores", None)
        
        # Use either the prop scores from the neural or the logistic regression
        # or both!
        if (nn_p_scores is not None) and kwargs.get("nn_p_scores_with_logistic", False):
            print("Using neural net and logistic p scores")
            matching_data = np.hstack([covariate_data, nn_p_scores.reshape(-1, 1),
                                       logistic_p_scores.reshape(-1, 1)])
            
        elif (nn_p_scores is not None):
            print("Using neural net  p scores")
            matching_data = np.hstack([covariate_data, nn_p_scores.reshape(-1, 1)])
        else:
            print("Using logistic p scores")
            matching_data = np.hstack([covariate_data, logistic_p_scores.reshape(-1, 1)])
             
    else:
        print("Not using prop scores")
        matching_data = covariate_data
    
    # Allow evaluation of balance on custom vars
    balance_vars = kwargs.get("balance_vars", None)
    if balance_vars is None:
        print("Evaluating balance on matching covars")
        balance_vars = covariate_data
    else:
        print("Evaluating balance on custom vars")
    
    # Run GenMatch
    print("Matching data shape:", matching_data.shape)
    print("Balance vars shape:", balance_vars.shape)
    
    start = time()
    gen_out = matching.GenMatch(
        Tr=IntVector(assignments),
        X=matching_data,
        BalanceMatrix=balance_vars,
        wait_generations=5,
        pop_size=1000,
        print_level=0,
        cluster=cl)
    
    print("GenMatch Time: ", time() - start)
    
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=matching_data,
        replace=True,
        Weight_matrix=gen_out)
    
    return np.array(match_out.rx2("est").rx(1,1))[0], \
            np.array(gen_out.rx2("value")), \
            np.diag(np.array(gen_out.rx2("Weight.matrix")))

In [9]:
# DEBUG
# est = logistic_prop_matching_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

In [10]:
# DEBUG
# est = genmatch_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

### C. Data Interconnect

CSV files are used to pass information between this file and the Neural Network files. The code below defines helper functions which save data generated in this file to CSVs for neural net training and  functions which read in the processed data which results from the training. 

In [16]:
RAW_DATA_DIR = "../Data/NSW/Raw"
PROCESSED_DATA_DIR = "../Data/NSW/Processed/"
VAE = "VAE/"
REG = "Regression/"

In [110]:
# Create assignment, outcome and covar data files based on
# the data available at http://users.nber.org/~rdehejia/data/nswdata2.html
def process_raw_nsw_files():
    treated = np.loadtxt(RAW_DATA_DIR + "/nswre74_treated.txt", delimiter="  ", dtype=str)[:, 1:].astype(float)
    control = np.loadtxt(RAW_DATA_DIR + "/nswre74_control.txt", delimiter="  ", dtype=str)[:, 1:].astype(float)
    cps_control = np.loadtxt(RAW_DATA_DIR + "/cps_controls.txt", delimiter="  ", dtype=str)[:, 1:].astype(float)

    experimental_data = np.vstack([treated, control])
    experimental_assignments = experimental_data[:, 0]
    experimental_outcomes = experimental_data[:, 9]
    experimental_covars = experimental_data[:, 1:9]

    exp_effect = np.mean(experimental_outcomes[experimental_assignments==1]) - \
        np.mean(experimental_outcomes[experimental_assignments==0])
    print("Exp treat effect:", exp_effect)
    assert(int(exp_effect) == 1794)
    
    np.savetxt(RAW_DATA_DIR + "nsw74_experiment_assignments.csv", experimental_assignments, delimiter=",")
    np.savetxt(RAW_DATA_DIR + "nsw74_experiment_outcomes.csv", experimental_outcomes, delimiter=",")
    np.savetxt(RAW_DATA_DIR + "nsw74_experiment_covars.csv", experimental_covars, delimiter=",")
    
    
    observational_data = np.vstack([treated, cps_control])
    observational_assignments = observational_data[:, 0]
    observational_outcomes = observational_data[:, 9]
    observational_covars = observational_data[:, 1:9]
    
    obs_effect = np.mean(observational_outcomes[observational_assignments==1]) - \
        np.mean(observational_outcomes[observational_assignments==0])
    print("Obs treat effect:", obs_effect)
    
    np.savetxt(RAW_DATA_DIR + "nsw74_all_assignments.csv", observational_assignments, delimiter=",")
    np.savetxt(RAW_DATA_DIR + "nsw74_all_outcomes.csv", observational_outcomes, delimiter=",")
    np.savetxt(RAW_DATA_DIR + "nsw74_all_covars.csv", observational_covars, delimiter=",")
    
    return exp_effect

# Retrieve processed and unprocessed data from files in order to run experiments. 
def get_data_from_file(loss_type=None, nn_p_regression=False, original_mode=False):
    
    original_covariates = np.loadtxt(RAW_DATA_DIR + "nsw74_all_covars.csv", delimiter=",")
    outcomes = np.loadtxt(RAW_DATA_DIR + "nsw74_all_outcomes.csv", delimiter=",")
    assignments = np.loadtxt(RAW_DATA_DIR + "nsw74_all_assignments.csv", delimiter=",")
    
    extra_data = {}
    if not (loss_type or nn_p_regression or original_mode):
        raise Exception("Invalid config. Need loss type, p regression, or original mode option")
    
    if not original_mode:
        if loss_type:
            if loss_type in ["reconstruction", "sparsity"]:
                covariate_file_name = PROCESSED_DATA_DIR + "nsw74_all_covars_{}.csv".format(loss_type)
                covariates = np.loadtxt(covariate_file_name, delimiter=",")

            elif loss_type in ["vae"]:
                covariate_file_name = PROCESSED_DATA_DIR + VAE + "nsw74_all_covars.csv"
                covariates_with_std = np.loadtxt(covariate_file_name, delimiter=",")

                # Split means and covariance
                column_count = covariates_with_std.shape[1]
                covar_marker = int(column_count/2)

                covariates = covariates_with_std[:, :covar_marker]
                extra_data["covariate_covariance"] = covariates_with_std[:, covar_marker:]

            if not nn_p_regression:
                # If we aren't accessing NN prop scores, return now.
                return assignments, outcomes, covariates, original_covariates, extra_data

        if nn_p_regression:
            reg_file_name = PROCESSED_DATA_DIR + REG + "nsw74_all_covars.csv"
            regression_prop_scores = np.loadtxt(reg_file_name, delimiter=",")

            extra_data["nn_p_scores"] = regression_prop_scores

            if loss_type:
                return assignments, outcomes, covariates, original_covariates, extra_data
    
    # Original data path
    return assignments, outcomes, original_covariates, extra_data

In [111]:
# Write data files for 1000 runs all models
# Careful with this, it writes ~3GB of data. 
process_files = True
if process_files:
    nsw_effect = process_raw_nsw_files()

Exp treat effect: 1794.342404270271
Obs treat effect: -8497.516142636992


In [112]:
assignments, outcomes, covariates, original_covariates, extra_data = \
get_data_from_file(loss_type="reconstruction")
print(covariates.shape)
covariates

(445, 4)


array([[-1.75889225e+01,  8.37841892e+00, -4.42144279e+01,
         1.55335531e+01],
       [-9.71539879e+00,  4.70030975e+00, -2.46371212e+01,
         9.08783340e+00],
       [-1.19481726e+01,  7.95542097e+00, -3.31328430e+01,
         1.31192455e+01],
       ...,
       [-1.98494873e+03, -7.48577148e+03,  9.15058887e+03,
         4.77715430e+03],
       [ 6.01781201e+03, -1.59553242e+04,  1.08096357e+04,
         2.18314185e+03],
       [ 4.02309717e+03, -1.76349434e+04,  1.40734307e+04,
         4.13518945e+03]])

Utilities to store and retrive pickled results dictionaries. This allows us to persist and sync results across different machines/processes.

In [27]:
def store_results_dict(results, name):
    pickle.dump(results, open("../Results/NSW/{}.p".format(name), "wb" ))
    
def retrieve_results_dict(name):
    try:
        return pickle.load(open( "../Results/NSW/{}.p".format(name), "rb" ))
    except Exception as e:
        return None

### D. Experiment Runner Code

This code is largely a duplication of the the code in the Simulations file, but simplified for a single run on the NSW dataset.

#### Single Simulation

In [28]:
# Wrapper function for all three of the matching methods above. This allows the matching function
# to be passed into the experiment running code without concern over the method API.
def get_estimate(outcomes, assignments, covar_data, method, *args, **kwargs):
    return method(outcomes, assignments, covar_data, *args, **kwargs)

In [106]:
def run_simulation(estimator=logistic_prop_matching_est,
                   store_name=None,
                   overwrite=False,
                   verbose=True,
                   *args, **kwargs):
    
    global gm_warnings
    gm_warnings = True
    
    result_data = None
    
    if store_name:
        result_data = retrieve_results_dict(store_name)
    
    if overwrite or (not result_data):
        print("No valid, existant results found. Beggining trial.\n")
    
        # Prepare config for matching estimators
        balance_vars = None
        propensity_vars = None
        extra_data = {}

        # Prepare data for matching
        loss_type = kwargs.get("loss_type", None)
        nn_p_regression = kwargs.get("nn_p_regression", None)
        original_mode = kwargs.get("original_mode", None)

        if not (loss_type or nn_p_regression or original_mode):
            raise Exception("Must supply loss type or p regression option to read from files")

        if loss_type:
            assignments, outcomes, covar_data, original_covars, extra_data = \
                get_data_from_file(loss_type=loss_type, nn_p_regression=nn_p_regression)

            if kwargs.get("evaluate_on_original_covars", False):
                balance_vars=original_covars

            if kwargs.get("propensity_on_original_covars", False):
                propensity_vars=original_covars

        else:
            assignments, outcomes, covar_data, extra_data =\
                get_data_from_file(
                    loss_type=loss_type,
                    nn_p_regression=nn_p_regression,
                    original_mode=original_mode)

        # Run matching
        result_tuple  = get_estimate(outcomes,
                                  assignments,
                                  covar_data,
                                  estimator,
                                  balance_vars=balance_vars,
                                  propensity_vars=propensity_vars,
                                  *args,
                                  **extra_data,
                                  **kwargs)
            
        result = result_tuple[0]
        
        bias = np.abs((nsw_effect-result)/nsw_effect * 100)

        result_data = {"result": result, "Bias": bias}
        
        if estimator == genmatch_est:
            result_data["fitnesses"] = result_tuple[1]
            result_data["weights"] = result_tuple[2]
            
            if verbose:
                print("Fitnesses: ", result_data["fitnesses"][:10])
                print("Weights: ",  result_data["weights"])
        
        if store_name:
            store_results_dict(result_data, store_name)
    else:   
        if verbose:
            print("Displaying cached results.\n")
    
    if verbose:
        print("Bias", result_data["Bias"])
        print("Estimated effect ${}".format(result_data["result"]))
        
    return result_data

In [ ]:
run_simulation(
    estimator=genmatch_est,
    store_name="original_mode",
    original_mode=True,
    overwrite=True,
    verbose=True)

No valid, existant results found. Beggining trial.



#### AE with only Reconstruction Loss


#### Config 1
Pure reconstruction with propensity score estimates

In [ ]:
run_simulation(
    estimator=genmatch_est,
    loss_type="reconstruction",
    store_name="AE_reconstruction_plain",
    verbose=True)

#### Config 2
Pure reconstruction *without* propensity score estimates

In [39]:
run_test_battery(
    est=genmatch_est,
    runs=ae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="reconstruction",
    results_subfolder="AE/Reconstruction/nopropscores",
    genmatch_with_prop_scores=False,
    verbosity=2)

Results File: AE/Reconstruction/nopropscores/est_genmatch_est_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  0.5388832593158581
1 RMSE:  0.07368715901276338 

Model:  B_add_mild_nlin
1 Bias:  3.276210486566693
1 RMSE:  0.0816027177764639 

Model:  C_add_mod_nlin
1 Bias:  0.6999435126924826
1 RMSE:  0.07554576764697919 

Model:  D_mild_nadd_lin
1 Bias:  4.937316461303445
1 RMSE:  0.08274988726545493 

Model:  E_mild_nadd_mild_nlin
1 Bias:  7.481250991590314
1 RMSE:  0.08478344012915207 

Model:  F_mod_nadd_lin
1 Bias:  0.9411372663771971
1 RMSE:  0.07537800943456885 

Model:  G_mod_nadd_mod_nlin
1 Bias:  0.8048814215327594
1 RMSE:  0.07788164426593222 



#### Config 3
Pure reconstruction, evaluating balance on uncompressed data, *without* propensity score.

In [40]:
run_test_battery(
    est=genmatch_est,
    runs=ae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="reconstruction",
    results_subfolder="AE/Reconstruction/evalonoriginal",
    evaluate_on_original_covars=True,
    genmatch_with_prop_scores=False,
    verbosity=2)

Results File: AE/Reconstruction/evalonoriginal/est_genmatch_est_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  1.552233121749752
1 RMSE:  0.06527597429814515 

Model:  B_add_mild_nlin
1 Bias:  3.333034582793069
1 RMSE:  0.06967542349832183 

Model:  C_add_mod_nlin
1 Bias:  0.6790984831418302
1 RMSE:  0.06579060225122386 

Model:  D_mild_nadd_lin
1 Bias:  4.3816613069158254
1 RMSE:  0.07068863465516317 

Model:  E_mild_nadd_mild_nlin
1 Bias:  6.876422257502002
1 RMSE:  0.07278407746587175 

Model:  F_mod_nadd_lin
1 Bias:  2.58611002166168
1 RMSE:  0.06981215022358879 

Model:  G_mod_nadd_mod_nlin
1 Bias:  0.502204523397222
1 RMSE:  0.0691460613259185 



#### Config 4
Pure reconstruction *with* propensity score derived from uncompressed data. Evaluating on uncompressed.

In [41]:
run_test_battery(
    est=genmatch_est,
    runs=ae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="reconstruction",
    results_subfolder="AE/Reconstruction/evalonoriginal_withp",
    evaluate_on_original_covars=True,
    propensity_on_original_covars=True,
    verbosity=2)

Results File: AE/Reconstruction/evalonoriginal_withp/est_genmatch_est_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  0.7067484388360161
1 RMSE:  0.03786592930152072 

Model:  B_add_mild_nlin
1 Bias:  0.13339724242402798
1 RMSE:  0.043485078136632285 

Model:  C_add_mod_nlin
1 Bias:  0.11494109318830585
1 RMSE:  0.050248376512268364 

Model:  D_mild_nadd_lin
1 Bias:  1.9150254839541625
1 RMSE:  0.04117176894231486 

Model:  E_mild_nadd_mild_nlin
1 Bias:  3.1961874905224175
1 RMSE:  0.04268150513623945 

Model:  F_mod_nadd_lin
1 Bias:  1.8485395850150468
1 RMSE:  0.04785803044605611 

Model:  G_mod_nadd_mod_nlin
1 Bias:  0.18265461358981974
1 RMSE:  0.053080370063081465 



#### Config 5
Pure reconstruction *with* propensity score derived from uncompressed data. Evaluating on compressed.

In [42]:
run_test_battery(
    est=genmatch_est,
    runs=ae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="reconstruction",
    results_subfolder="AE/Reconstruction/withp",
    evaluate_on_original_covars=False,
    propensity_on_original_covars=True,
    verbosity=2)

Results File: AE/Reconstruction/withp/est_genmatch_est_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  1.373322079710853
1 RMSE:  0.048648363590301766 

Model:  B_add_mild_nlin
1 Bias:  0.18520053724370186
1 RMSE:  0.053997099576098474 

Model:  C_add_mod_nlin
1 Bias:  0.6021376702766212
1 RMSE:  0.05363450514544961 

Model:  D_mild_nadd_lin
1 Bias:  1.9725293478895758
1 RMSE:  0.05146490692396631 

Model:  E_mild_nadd_mild_nlin
1 Bias:  4.154412436745519
1 RMSE:  0.05394638272616707 

Model:  F_mod_nadd_lin
1 Bias:  1.9623181573686646
1 RMSE:  0.055835868418105714 

Model:  G_mod_nadd_mod_nlin
1 Bias:  0.13880680510402485
1 RMSE:  0.060491851132907115 



#### AE with Reconstruction and Sparsity

#### Config 1

Sparse reconstruction *without* propensity score.

In [46]:
run_test_battery(
    est=genmatch_est,
    runs=50,
    n_samples=1000,
    from_files=True,
    loss_type="sparsity",
    results_subfolder="AE/Sparsity",
    genmatch_with_prop_scores=False,
    verbosity=2)

Results File: AE/Sparsity/est_genmatch_est_runs_50_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  8.526097159023248
1 RMSE:  0.1134592853375768 

Model:  B_add_mild_nlin
1 Bias:  4.131268935319159
1 RMSE:  0.1160868056178415 

Model:  C_add_mod_nlin
1 Bias:  8.430802732391273
1 RMSE:  0.1108407840160775 

Model:  D_mild_nadd_lin
1 Bias:  5.322330536939029
1 RMSE:  0.10460071276975336 

Model:  E_mild_nadd_mild_nlin
1 Bias:  0.8211996810856915
1 RMSE:  0.12143098844611416 

Model:  F_mod_nadd_lin
1 Bias:  13.889165268095837
1 RMSE:  0.13591469046446247 

Model:  G_mod_nadd_mod_nlin
1 Bias:  6.744962783709057
1 RMSE:  0.10264505172208324 



#### Config 2

Sparse with prop scores

In [47]:
run_test_battery(
    est=genmatch_est,
    runs=50,
    n_samples=1000,
    from_files=True,
    loss_type="sparsity",
    results_subfolder="AE/Sparsity/withp",
    genmatch_with_prop_scores=True,
    propensity_on_original_covars=True,
    verbosity=2)

Results File: AE/Sparsity/withp/est_genmatch_est_runs_50_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  1.6136292441155462
1 RMSE:  0.05547243408003388 

Model:  B_add_mild_nlin
1 Bias:  0.44513296886365444
1 RMSE:  0.0532783960077565 

Model:  C_add_mod_nlin
1 Bias:  0.4257899593481197
1 RMSE:  0.05914334516222583 

Model:  D_mild_nadd_lin
1 Bias:  0.4726712662232528
1 RMSE:  0.052783396792566356 

Model:  E_mild_nadd_mild_nlin
1 Bias:  2.583291458156751
1 RMSE:  0.06302520184923029 

Model:  F_mod_nadd_lin
1 Bias:  3.274417247927261
1 RMSE:  0.06293152300833735 

Model:  G_mod_nadd_mod_nlin
1 Bias:  0.695770947223328
1 RMSE:  0.06237666457453329 



#### Config 3
Sparse with prop scores, eval on original

In [48]:
run_test_battery(
    est=genmatch_est,
    runs=50,
    n_samples=1000,
    from_files=True,
    loss_type="sparsity",
    results_subfolder="AE/Sparsity/evalonoriginal_withp",
    genmatch_with_prop_scores=True,
    propensity_on_original_covars=True,
    evaluate_on_original_covars=True,
    verbosity=2)

Results File: AE/Sparsity/evalonoriginal_withp/est_genmatch_est_runs_50_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  3.0052785609630153
1 RMSE:  0.05135956480780116 

Model:  B_add_mild_nlin
1 Bias:  0.514080145976968
1 RMSE:  0.043153932448751264 

Model:  C_add_mod_nlin
1 Bias:  0.9633292625895341
1 RMSE:  0.04668919693650328 

Model:  D_mild_nadd_lin
1 Bias:  0.30727483094548486
1 RMSE:  0.05212186169768896 

Model:  E_mild_nadd_mild_nlin
1 Bias:  2.7111127095297336
1 RMSE:  0.05075685049205623 

Model:  F_mod_nadd_lin
1 Bias:  0.43743334386364874
1 RMSE:  0.04714358098168887 

Model:  G_mod_nadd_mod_nlin
1 Bias:  0.9975102646443165
1 RMSE:  0.05399429284681247 



### Results Printout

In [50]:
gm_results = retrieve_results_dict(gm_combined_name)
lin_results = retrieve_results_dict(linear_combined_name)

gm_ae_recon_results = retrieve_results_dict(
    get_store_name("AE/Reconstruction", assignment_model_names, genmatch_est, ae_runs, 1000))

gm_ae_recon_no_prop_score_results = retrieve_results_dict(
    get_store_name("AE/Reconstruction/nopropscores", assignment_model_names, genmatch_est, ae_runs, 1000))

gm_ae_recon_original_fitness_results = retrieve_results_dict(
    get_store_name("AE/Reconstruction/evalonoriginal", assignment_model_names, genmatch_est, ae_runs, 1000))

gm_ae_recon_original_fitness_with_prop_score_results = retrieve_results_dict(
    get_store_name("AE/Reconstruction/evalonoriginal_withp", assignment_model_names, genmatch_est, ae_runs, 1000))

gm_ae_recon_with_prop_score_results = retrieve_results_dict(
    get_store_name("AE/Reconstruction/withp", assignment_model_names, genmatch_est, ae_runs, 1000))


###

gm_ae_sparse_results = retrieve_results_dict(
    get_store_name("AE/Sparsity", assignment_model_names, genmatch_est, 50, 1000))

gm_ae_sparse_original_fitness_with_prop_score_results = retrieve_results_dict(
    get_store_name("AE/Sparsity/evalonoriginal_withp", assignment_model_names, genmatch_est, 50, 1000))

gm_ae_sparse_with_prop_score_results = retrieve_results_dict(
    get_store_name("AE/Sparsity/withp", assignment_model_names, genmatch_est, 50, 1000))



results = {
    "Linear": lin_results,
    "GenMatch": gm_results,
    "GenMatch AE Recon": gm_ae_recon_results,
    "GenMatch AE Recon, No P Score": gm_ae_recon_no_prop_score_results,
    "GenMatch AE Recon, Org. Fitness": gm_ae_recon_original_fitness_results,
    "GenMatch AE Recon, Org. Fitness, With P": gm_ae_recon_original_fitness_with_prop_score_results,
    "GenMatch AE Recon, With P": gm_ae_recon_with_prop_score_results,
    ###
    "GenMatch AE Sparse": gm_ae_sparse_results,
    "GenMatch AE Sparse, Org. Fitness, With P": gm_ae_sparse_original_fitness_with_prop_score_results,
    "GenMatch AE Sparse, With P": gm_ae_sparse_with_prop_score_results,
}

for model in assignment_model_names:
    print(model, "\n")
    for matching in results.keys():
        print(matching)
        print("RMSE:", np.round(results[matching][model]["RMSE"], 4), "Bias:",
              np.round(results[matching][model]["Bias"], 4))
        
    print("==============")
    print()
    

A_add_lin 

Linear
RMSE: 0.0731 Bias: 0.0459
GenMatch
RMSE: 0.0416 Bias: 5.5586
GenMatch AE Recon
RMSE: 0.0794 Bias: 0.4983
GenMatch AE Recon, No P Score
RMSE: 0.0737 Bias: 0.5389
GenMatch AE Recon, Org. Fitness
RMSE: 0.0653 Bias: 1.5522
GenMatch AE Recon, Org. Fitness, With P
RMSE: 0.0379 Bias: 0.7067
GenMatch AE Recon, With P
RMSE: 0.0486 Bias: 1.3733
GenMatch AE Sparse
RMSE: 0.1135 Bias: 8.5261
GenMatch AE Sparse, Org. Fitness, With P
RMSE: 0.0514 Bias: 3.0053
GenMatch AE Sparse, With P
RMSE: 0.0555 Bias: 1.6136

B_add_mild_nlin 

Linear
RMSE: 0.0659 Bias: 3.1844
GenMatch
RMSE: 0.038 Bias: 4.3099
GenMatch AE Recon
RMSE: 0.0825 Bias: 5.6199
GenMatch AE Recon, No P Score
RMSE: 0.0816 Bias: 3.2762
GenMatch AE Recon, Org. Fitness
RMSE: 0.0697 Bias: 3.333
GenMatch AE Recon, Org. Fitness, With P
RMSE: 0.0435 Bias: 0.1334
GenMatch AE Recon, With P
RMSE: 0.054 Bias: 0.1852
GenMatch AE Sparse
RMSE: 0.1161 Bias: 4.1313
GenMatch AE Sparse, Org. Fitness, With P
RMSE: 0.0432 Bias: 0.5141
GenMatc

### E. VAE

In [51]:
vae_runs = 200

#### 1. GenMatch in latent space

#### Config 1
Genmatch without propensity score

In [52]:
run_test_battery(
    est=genmatch_est,
    runs=vae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="vae",
    results_subfolder="VAE/",
    genmatch_with_prop_scores=False,
    verbosity=2)

Results File: VAE//est_genmatch_est_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  7.312201130997685
1 RMSE:  0.05295933266800956 

Model:  B_add_mild_nlin
1 Bias:  7.214011335197719
1 RMSE:  0.052447212539800586 

Model:  C_add_mod_nlin
1 Bias:  5.105324395918432
1 RMSE:  0.04918834245145284 

Model:  D_mild_nadd_lin
1 Bias:  5.1043386948034275
1 RMSE:  0.04855458362958724 

Model:  E_mild_nadd_mild_nlin
1 Bias:  3.875474594602394
1 RMSE:  0.04781873913431988 

Model:  F_mod_nadd_lin
1 Bias:  8.549881888333742
1 RMSE:  0.0572041703159975 

Model:  G_mod_nadd_mod_nlin
1 Bias:  4.392239501605347
1 RMSE:  0.04799559601072314 



#### Config 2

GenMatch with propensity on original covars, evaluating on latent. 

In [53]:
run_test_battery(
    est=genmatch_est,
    runs=vae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="vae",
    results_subfolder="VAE/withp",
    genmatch_with_prop_scores=True,
    propensity_on_original_covars=True,
    verbosity=2)

Results File: VAE/withp/est_genmatch_est_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  2.5395433059520007
1 RMSE:  0.04808528784042503 

Model:  B_add_mild_nlin
1 Bias:  2.04490054506496
1 RMSE:  0.04660732524183032 

Model:  C_add_mod_nlin
1 Bias:  0.15073221258237324
1 RMSE:  0.0507555360190453 

Model:  D_mild_nadd_lin
1 Bias:  0.3606620636051126
1 RMSE:  0.04962363133543826 

Model:  E_mild_nadd_mild_nlin
1 Bias:  1.5379434666280805
1 RMSE:  0.0499287081523966 

Model:  F_mod_nadd_lin
1 Bias:  3.1169056061108686
1 RMSE:  0.05503239603006674 

Model:  G_mod_nadd_mod_nlin
1 Bias:  1.3989182232673525
1 RMSE:  0.05336476463717196 



#### Config 3

GenMatch with propensity on original covars, evaluating balance on original covars.

In [54]:
run_test_battery(
    est=genmatch_est,
    runs=vae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="vae",
    results_subfolder="VAE/evalonoriginal_withp",
    genmatch_with_prop_scores=True,
    propensity_on_original_covars=True,
    evaluate_on_original_covars=True,
    verbosity=2)

Results File: VAE/evalonoriginal_withp/est_genmatch_est_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  0.8517170004947486
1 RMSE:  0.04701381917058319 

Model:  B_add_mild_nlin
1 Bias:  0.3672756443601539
1 RMSE:  0.048237894709229706 

Model:  C_add_mod_nlin
1 Bias:  1.3369943583773656
1 RMSE:  0.053340455624368247 

Model:  D_mild_nadd_lin
1 Bias:  1.1463459806259935
1 RMSE:  0.04852571125820148 

Model:  E_mild_nadd_mild_nlin
1 Bias:  3.383983114942237
1 RMSE:  0.05382206092404716 

Model:  F_mod_nadd_lin
1 Bias:  1.2160734246097318
1 RMSE:  0.055325910561995914 

Model:  G_mod_nadd_mod_nlin
1 Bias:  2.619943937710873
1 RMSE:  0.052742346254803 



#### 2. Mahalanobis Distance

Use the latent space means and standard deviations to try direct matching. 

#### Config 1

MD distance plain

In [55]:
run_test_battery(
    est=mahalanobis_matching,
    runs=vae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="vae",
    results_subfolder="VAE/md",
    verbosity=2)

Results File: VAE/md/est_mahalanobis_matching_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  7.544441012550429
1 RMSE:  0.05138754558193052 

Model:  B_add_mild_nlin
1 Bias:  7.298749820510931
1 RMSE:  0.050962216786474376 

Model:  C_add_mod_nlin
1 Bias:  3.9777874804774442
1 RMSE:  0.04374998484853666 

Model:  D_mild_nadd_lin
1 Bias:  5.180281858062211
1 RMSE:  0.0462263853072062 

Model:  E_mild_nadd_mild_nlin
1 Bias:  4.192882095179809
1 RMSE:  0.04551575417194539 

Model:  F_mod_nadd_lin
1 Bias:  7.983947334265511
1 RMSE:  0.05533750312001229 

Model:  G_mod_nadd_mod_nlin
1 Bias:  4.230594789867654
1 RMSE:  0.047919920965010125 



#### Config 2
MD distance with propensity scores

In [56]:
run_test_battery(
    est=mahalanobis_matching,
    runs=vae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="vae",
    results_subfolder="VAE/md_withp",
    md_with_prop_scores=True,
    propensity_on_original_covars=True,
    verbosity=2)

Results File: VAE/md_withp/est_mahalanobis_matching_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  7.57902532597313
1 RMSE:  0.05149514462878567 

Model:  B_add_mild_nlin
1 Bias:  7.299220691076224
1 RMSE:  0.050977018369883775 

Model:  C_add_mod_nlin
1 Bias:  3.9412419310526183
1 RMSE:  0.043646522766075474 

Model:  D_mild_nadd_lin
1 Bias:  5.149583862026001
1 RMSE:  0.04606738315619503 

Model:  E_mild_nadd_mild_nlin
1 Bias:  4.1849816542073155
1 RMSE:  0.04538368034996612 

Model:  F_mod_nadd_lin
1 Bias:  7.9760975724759895
1 RMSE:  0.05528390984921817 

Model:  G_mod_nadd_mod_nlin
1 Bias:  4.215663518355213
1 RMSE:  0.04782197261829298 



#### Config 3

Use the Bhattacharyya Distance distance which finds distributional overlap for normal distributions.

In [57]:
run_test_battery(
    est=mahalanobis_matching,
    runs=vae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="vae",
    results_subfolder="VAE/Z2/bhat",
    distance_metric="bhat",
    verbosity=2)

Results File: VAE/Z2/bhat/est_mahalanobis_matching_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  7.556229902669169
1 RMSE:  0.050628029614782195 

Model:  B_add_mild_nlin
1 Bias:  7.318992673380274
1 RMSE:  0.050024202084690994 

Model:  C_add_mod_nlin
1 Bias:  4.181261410314277
1 RMSE:  0.04285994072768112 

Model:  D_mild_nadd_lin
1 Bias:  5.085225497432111
1 RMSE:  0.0452803013165698 

Model:  E_mild_nadd_mild_nlin
1 Bias:  4.142427330949168
1 RMSE:  0.045729912752522466 

Model:  F_mod_nadd_lin
1 Bias:  8.097072410798255
1 RMSE:  0.0545518692276087 

Model:  G_mod_nadd_mod_nlin
1 Bias:  4.523303488528463
1 RMSE:  0.04781288827208273 



### F. Neural Propensity Score Regression

In [30]:
reg_ae_runs = 200

In [72]:
run_test_battery(
    est=logistic_prop_matching_est,
    runs=200,
    n_samples=1000,
    from_files=True,
    results_subfolder="NN",
    nn_p_regression=True
)

Results File: NN/est_logistic_prop_matching_est_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  2.516970645424258
1 RMSE:  0.08208114416711368 

Model:  B_add_mild_nlin
1 Bias:  3.483748622303001
1 RMSE:  0.08543882017480643 

Model:  C_add_mod_nlin
1 Bias:  5.00918193487844
1 RMSE:  0.10057195625252761 

Model:  D_mild_nadd_lin
1 Bias:  4.505665739350313
1 RMSE:  0.09322603726388229 

Model:  E_mild_nadd_mild_nlin
1 Bias:  5.654346511047072
1 RMSE:  0.08809153862302047 

Model:  F_mod_nadd_lin
1 Bias:  1.3475296921836275
1 RMSE:  0.09338027635325147 

Model:  G_mod_nadd_mod_nlin
1 Bias:  8.98239908298724
1 RMSE:  0.12032750964509623 



In [57]:
run_test_battery(
    est=genmatch_est,
    runs=250,
    n_samples=1000,
    from_files=True,
    results_subfolder="NN/genmatch",
    genmatch_with_prop_scores=True,
    nn_p_regression=True,
    verbosity=2)

Results File: NN/genmatch/est_genmatch_est_runs_250_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  4.29807544319546
1 RMSE:  0.03924093815095593 

Model:  B_add_mild_nlin
1 Bias:  5.027476827504075
1 RMSE:  0.041036686487701785 

Model:  C_add_mod_nlin
1 Bias:  2.1088702260847136
1 RMSE:  0.04341309335599729 

Model:  D_mild_nadd_lin
1 Bias:  2.3862953259334945
1 RMSE:  0.03882568477146027 

Model:  E_mild_nadd_mild_nlin
1 Bias:  1.01074209695447
1 RMSE:  0.03792983147379567 

Model:  F_mod_nadd_lin
1 Bias:  4.655858061545212
1 RMSE:  0.04384773084167265 

Model:  G_mod_nadd_mod_nlin
1 Bias:  2.032590286077631
1 RMSE:  0.04310322815049054 



### Regression + AE

In [35]:
run_test_battery(
    est=genmatch_est,
    runs=reg_ae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="reconstruction",
    results_subfolder="NN/genmatch_reconstruction",
    genmatch_with_prop_scores=True,
    nn_p_regression=True,
    verbosity=2)

Results File: NN/genmatch_reconstruction/est_genmatch_est_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  0.529673171829923
1 RMSE:  0.05386042530009443 

Model:  B_add_mild_nlin
1 Bias:  1.536696262108329
1 RMSE:  0.05679903685019429 

Model:  C_add_mod_nlin
1 Bias:  0.08182277851303908
1 RMSE:  0.06297264676850725 

Model:  D_mild_nadd_lin
1 Bias:  3.6659780883960047
1 RMSE:  0.05920459006176178 

Model:  E_mild_nadd_mild_nlin
1 Bias:  4.789360578078707
1 RMSE:  0.058677011907493345 

Model:  F_mod_nadd_lin
1 Bias:  0.5585211201953151
1 RMSE:  0.05941723315251356 

Model:  G_mod_nadd_mod_nlin
1 Bias:  0.21314704713027943
1 RMSE:  0.06181632295148564 



In [36]:
run_test_battery(
    est=genmatch_est,
    runs=reg_ae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="reconstruction",
    results_subfolder="NN/genmatch_reconstruction_evalonoriginal",
    genmatch_with_prop_scores=True,
    nn_p_regression=True,
    evaluate_on_original_covars=True,
    verbosity=2)

Results File: NN/genmatch_reconstruction_evalonoriginal/est_genmatch_est_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  0.05733422862098678
1 RMSE:  0.04460639113827511 

Model:  B_add_mild_nlin
1 Bias:  0.3416270354118921
1 RMSE:  0.04337636216599876 

Model:  C_add_mod_nlin
1 Bias:  0.17896723806440895
1 RMSE:  0.055350444444338454 

Model:  D_mild_nadd_lin
1 Bias:  2.2316810121202617
1 RMSE:  0.04387889187316208 

Model:  E_mild_nadd_mild_nlin
1 Bias:  3.344728110847269
1 RMSE:  0.04439170278043582 

Model:  F_mod_nadd_lin
1 Bias:  0.23455257555113562
1 RMSE:  0.04949492394802052 

Model:  G_mod_nadd_mod_nlin
1 Bias:  0.007101031185222269
1 RMSE:  0.05363245105383774 



### Reconstruction + VAE

In [37]:
run_test_battery(
    est=genmatch_est,
    runs=reg_ae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="vae",
    results_subfolder="NN/genmatch_vae",
    genmatch_with_prop_scores=True,
    nn_p_regression=True,
    verbosity=2)

Results File: NN/genmatch_vae/est_genmatch_est_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  1.8614017258002897
1 RMSE:  0.046627158814633134 

Model:  B_add_mild_nlin
1 Bias:  1.0623641880990218
1 RMSE:  0.04780889865562421 

Model:  C_add_mod_nlin
1 Bias:  1.1022105885256166
1 RMSE:  0.05763152334980864 

Model:  D_mild_nadd_lin
1 Bias:  0.8832082725827941
1 RMSE:  0.05134149227345619 

Model:  E_mild_nadd_mild_nlin
1 Bias:  1.180599024428345
1 RMSE:  0.051764207766344696 

Model:  F_mod_nadd_lin
1 Bias:  3.371783090461461
1 RMSE:  0.06068408190931019 

Model:  G_mod_nadd_mod_nlin
1 Bias:  0.6670186230576475
1 RMSE:  0.05552969888840167 



In [38]:
run_test_battery(
    est=genmatch_est,
    runs=reg_ae_runs,
    n_samples=1000,
    from_files=True,
    loss_type="vae",
    results_subfolder="NN/genmatch_vae_evaloriginal",
    genmatch_with_prop_scores=True,
    nn_p_regression=True,
    evaluate_on_original_covars=True,
    verbosity=2)

Results File: NN/genmatch_vae_evaloriginal/est_genmatch_est_runs_200_n_1000
Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  0.025656542980655707
1 RMSE:  0.04467389196112116 

Model:  B_add_mild_nlin
1 Bias:  0.7731961580198056
1 RMSE:  0.05205840412610209 

Model:  C_add_mod_nlin
1 Bias:  0.9092655458551755
1 RMSE:  0.05781784981206415 

Model:  D_mild_nadd_lin
1 Bias:  3.038828666389273
1 RMSE:  0.05245779974299758 

Model:  E_mild_nadd_mild_nlin
1 Bias:  2.872895222216922
1 RMSE:  0.05685137200754044 

Model:  F_mod_nadd_lin
1 Bias:  1.2299437114421792
1 RMSE:  0.05633433796082689 

Model:  G_mod_nadd_mod_nlin
1 Bias:  1.0446758178775761
1 RMSE:  0.06328368505167929 

